# Import data from Smoothcomp 
https://svenskjudo.smoothcomp.com


In [57]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

## Identify all pages for an event

In [58]:


events = list()
events.append({"name": "Skåneserien 3 2021 Skurup", "url": "https://svenskjudo.smoothcomp.com/sv/event/3880/schedule/matchlist"})
events.append({"name": "Skåneserien 4 2021 Staffanstorp", "url": "https://svenskjudo.smoothcomp.com/sv/event/5516/schedule/matchlist"})
events.append({"name": "Skåneserien 1 2022 Skurup", "url": "https://svenskjudo.smoothcomp.com/sv/event/6424/schedule/matchlist"})
events.append({"name": "Skåneserien 2 2022 Skurup", "url": "https://svenskjudo.smoothcomp.com/sv/event/7314/schedule/matchlist"})
events.append({"name": "Skåneserien 3 2022 Knislinge", "url": "https://svenskjudo.smoothcomp.com/sv/event/8558/schedule/matchlist"})
events.append({"name": "Skåneserien 4 2022 Staffanstorp", "url": "https://svenskjudo.smoothcomp.com/sv/event/6838/schedule/matchlist"})
events.append({"name": "Skåneserien 2 2023 Knislinge", "url": "https://svenskjudo.smoothcomp.com/sv/event/9302/schedule/matchlist"})
events.append({"name": "Skåneserien 3 2023 Skurup", "url": "https://svenskjudo.smoothcomp.com/sv/event/10180/schedule/matchlist"})
events.append({"name": "Skåneserien 4 2023 Staffanstorp", "url": "https://svenskjudo.smoothcomp.com/sv/event/12871/schedule/matchlist"})
events.append({"name": "Skåneserien 1 2024 Staffanstorp", "url": "https://svenskjudo.smoothcomp.com/sv/event/14644/schedule/matchlist"})
events.append({"name": "Skåneserien 2 2024 Knislinge",    "url": "https://svenskjudo.smoothcomp.com/en/event/12968/schedule/matchlist"})

event= events[10]

pages = list()
response = requests.get(event["url"])
soup = BeautifulSoup(response.text, 'html.parser')
pagination = soup.find_all("ul", {"class": "pagination"})[0]
pages_count = len(pagination.findAll('li'))-2

for i in range(1,pages_count+1):
    pages.append(event["url"] + '?page=' + str(i))

pages_count

10

## Request and find all played matches from theses pages 

In [59]:
category_rows = list()
match_rows = list()

for page in pages:
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')

    matches_list = soup.find_all("div", {"class": "matches-list"})[0]

    category_rows += matches_list.findAll('div',  {"class": "category-row"} )
    match_rows += matches_list.findAll('div',  {"class": "match-row"} )


len(match_rows)

299

## Extract data into one table

In [60]:
table_matches = []
for i in range(len(match_rows)):
    tablerow = []
    category = category_rows[i].string.split('(')[0].strip()
    #competition = category_rows[i].string.split('(')[1].strip()[:-1].strip()
    competition = event["name"]

    spelare = match_rows[i].findAll('span',  {"class": "participant"} )
    spelare1 = spelare[0].text.replace("\n", ",").split(',')
    spelare2 = spelare[1].text.replace("\n", ",").split(',')
        
    if  (len(spelare[0].text.replace("\n", ",").split(','))==7): 
        table_matches.append([competition, category,spelare1[1],spelare1[2], spelare2[1],spelare2[2], "White", spelare1[1],spelare1[2], spelare2[1],spelare2[2] ])    
    else:
        table_matches.append([competition, category,spelare1[1],spelare1[2], spelare2[1],spelare2[2], "Blue",  spelare2[1],spelare2[2], spelare1[1],spelare1[2]])       
table_matches

[['Skåneserien 2 2024 Knislinge',
  'PU17-01',
  'Liam Weimer/churchill',
  'Anderslövs IF Judosektion',
  'Marc Sánchez Barkenhammar',
  'Näsets Budoklubb',
  'White',
  'Liam Weimer/churchill',
  'Anderslövs IF Judosektion',
  'Marc Sánchez Barkenhammar',
  'Näsets Budoklubb'],
 ['Skåneserien 2 2024 Knislinge',
  'U11 MIX-12',
  'Melaher Awet',
  'Knislinge Budoklubb',
  'Zola Wik',
  'GAK Enighet Judo',
  'White',
  'Melaher Awet',
  'Knislinge Budoklubb',
  'Zola Wik',
  'GAK Enighet Judo'],
 ['Skåneserien 2 2024 Knislinge',
  'U9 MIX-04',
  'Nicolas Jovanovic Lilliesköld',
  'Näsets Budoklubb',
  'John Lindberg',
  'Staffanstorps JK',
  'White',
  'Nicolas Jovanovic Lilliesköld',
  'Näsets Budoklubb',
  'John Lindberg',
  'Staffanstorps JK'],
 ['Skåneserien 2 2024 Knislinge',
  'U9 MIX-04',
  'Aylen Hernandez',
  'GAK Enighet Judo',
  'Ludvig Bergkvist',
  'Växjö Judoklubb',
  'White',
  'Aylen Hernandez',
  'GAK Enighet Judo',
  'Ludvig Bergkvist',
  'Växjö Judoklubb'],
 ['Skånes

## Create the dataframe

In [61]:
df_table_matches = pd.DataFrame(table_matches)
df_table_matches.columns = ['Competition', 'Category', 'White Name',     'White Club',        'Blue Name',         'Blue Club',          'Winner Color', 'Winner Name',       'Winner Club',      'Looser Name',   'Looser Club']
df_table_matches

,Competition,Category,White Name,White Club,Blue Name,Blue Club,Winner Color,Winner Name,Winner Club,Looser Name,Looser Club
0,Skåneserien 2 2024 Knislinge,PU17-01,Liam Weimer/churchill,Anderslövs IF Judosektion,Marc Sánchez Barkenhammar,Näsets Budoklubb,White,Liam Weimer/churchill,Anderslövs IF Judosektion,Marc Sánchez Barkenhammar,Näsets Budoklubb
1,Skåneserien 2 2024 Knislinge,U11 MIX-12,Melaher Awet,Knislinge Budoklubb,Zola Wik,GAK Enighet Judo,White,Melaher Awet,Knislinge Budoklubb,Zola Wik,GAK Enighet Judo
2,Skåneserien 2 2024 Knislinge,U9 MIX-04,Nicolas Jovanovic Lilliesköld,Näsets Budoklubb,John Lindberg,Staffanstorps JK,White,Nicolas Jovanovic Lilliesköld,Näsets Budoklubb,John Lindberg,Staffanstorps JK
3,Skåneserien 2 2024 Knislinge,U9 MIX-04,Aylen Hernandez,GAK Enighet Judo,Ludvig Bergkvist,Växjö Judoklubb,White,Aylen Hernandez,GAK Enighet Judo,Ludvig Bergkvist,Växjö Judoklubb
4,Skåneserien 2 2024 Knislinge,H+17-01,Nils Milhau,Knislinge Budoklubb,Abdulhadi Hamid,Kristianstad Judoklubb,Blue,Abdulhadi Hamid,Kristianstad Judoklubb,Nils Milhau,Knislinge Budoklubb
...,...,...,...,...,...,...,...,...,...,...,...
294,Skåneserien 2 2024 Knislinge,PU17-05,Joakim Mattisson,Karlskrona Judoklubb,Emilio Lindsjö,Österlen Budo klubb,Blue,Emilio Lindsjö,Österlen Budo klubb,Joakim Mattisson,Karlskrona Judoklubb
295,Skåneserien 2 2024 Knislinge,FU17-01,Minnah Börjeson,Ljungbyheds Judo och Kampsport,Hanna Duratovic,Anderslövs IF Judosektion,White,Minnah Börjeson,Ljungbyheds Judo och Kampsport,Hanna Duratovic,Anderslövs IF Judosektion
296,Skåneserien 2 2024 Knislinge,FU17-02,Nemi Thuresson,Växjö Judoklubb,Elvira Brandel,Växjö Judoklubb,Blue,Elvira Brandel,Växjö Judoklubb,Nemi Thuresson,Växjö Judoklubb
297,Skåneserien 2 2024 Knislinge,FU17-02,Eja Magnusson,Söderskogs IF,Maya-My Haider,Älmhults Judoklubb,White,Eja Magnusson,Söderskogs IF,Maya-My Haider,Älmhults Judoklubb


## Export current event to a csv file

In [62]:
df_table_matches.to_csv(event["name"] + '.csv', index=False, encoding='utf-8-sig')# cp1252  iso-8859-1 utf-8 